In [1]:
import sys
sys.path.append("..")
from models import BertForSequenceClassification

import datasets
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
from transformers import AutoTokenizer, AutoModel

/home/newdisk/jxh/anaconda/envs/sft2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
base_path = "../base/finbert/"
ckpt_path = "../outputs/bert_sentiment_90Acc.pt"
tokenizer = AutoTokenizer.from_pretrained(base_path)
bert = AutoModel.from_pretrained(base_path)
classifier = BertForSequenceClassification(bert, 2, True)
classifier.load_state_dict(torch.load(ckpt_path))

pred_mapper = {
    0: "NEGATIVE",
    1: "POSITIVE"
  }


<All keys matched successfully>

In [3]:
classifier

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [21]:
seqs = ["盛运环保2月13日晚间发布公告称，截至目前，共有37.48亿元到期债务未清偿。", "真好啊"]

tokens = tokenizer(seqs, return_tensors="pt",
                    padding=True, truncation=True, max_length=512)
classifier_outputs = classifier(tokens["input_ids"],tokens["attention_mask"]).squeeze(dim=0)
probability = torch.softmax(classifier_outputs, dim=-1).cpu().detach().numpy()
pred_labels = [pred_mapper[pred] for pred in classifier_outputs.argmax(dim=-1).cpu().detach().numpy()]
for i in range(len(seqs)):
    print(seqs[i],pred_labels[i],probability[i])

盛运环保2月13日晚间发布公告称，截至目前，共有37.48亿元到期债务未清偿。 NEGATIVE [0.9558886 0.0441114]
真好啊 POSITIVE [0.1256033  0.87439674]
